In [64]:
import pandas as pd
import numpy as np
import nibabel as nib
from os.path import join
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import ttest_ind


In [2]:
df_pacientes = pd.read_csv('Clasificados.csv')
df_atlas = pd.read_csv('Harvard_Oxford.csv')
dir_ = r'C:\Users\jesus\Work\MIA\Autopsia'

atlas_path = r'C:\Users\jesus\Work\repos\Master_IA_Sanidad\Modulo_3\Clases\Clase_Semana_29_CDSS\cortex_Harvard_Oxford_orig_atlas.nii.gz'
atlas_img = nib.load(atlas_path)
atlas_data = atlas_img.get_fdata()

In [33]:
df_atlas

,ROI_NAME,ROI_NUM,PROMEDIO,STD
0,Frontal_Pole,1,1.297428,0.046500
1,Insular_Cortex,2,1.318357,0.041654
2,Superior_Frontal_Gyrus,3,1.321448,0.064708
3,Middle_Frontal_Gyrus,4,1.407354,0.051632
4,Inferior_Frontal_Gyrus_pars_triangularis,5,1.356674,0.045377
5,Inferior_Frontal_Gyrus_pars_opercularis,6,1.415501,0.051962
6,Precentral_Gyrus,7,1.348035,0.044931
7,Temporal_Pole,8,0.965558,0.054513
8,Superior_Temporal_Gyrus_anterior_division,9,1.161459,0.045870
9,Superior_Temporal_Gyrus_posterior_division,10,1.280519,0.042944


In [4]:
for index_1, row_1 in df_pacientes.iterrows():
    
    id_subject = row_1['SUBJECT_ID']
    id_img = row_1['FDG_ID']
    subj_dir_name = id_subject + '_I' + str(id_img)

    img_path = join(dir_,subj_dir_name,'swfdg_normhist.hdr')
    img_ = nib.load(img_path)
    img_data = img_.get_fdata()
    
    for index_2, row_2 in df_atlas.iterrows():
        roi_name = row_2['ROI_NAME']
        roi_num = row_2['ROI_NUM']
        roi_promedio = row_2['PROMEDIO']
        roi_std = row_2['STD']
        
        roi_voxel = np.where(atlas_data == roi_num)
        val_patient = np.mean(img_data[roi_voxel])
        
        pat_roi_zscore = (val_patient - roi_promedio)/roi_std

        df_pacientes.loc[index_1,roi_name] = pat_roi_zscore
                

In [13]:
df_pacientes.to_csv('roi_values.csv')

In [6]:
df_ad = df_pacientes.loc[df_pacientes['Grupo']=='AD']
df_lewy = df_pacientes.loc[df_pacientes['Grupo']=='LEWY']

ad_pattern = np.zeros_like(atlas_data)
lewy_body_pattern = np.zeros_like(atlas_data)

for index_2, row_2 in df_atlas.iterrows():
    roi_num = row_2['ROI_NUM']
    roi_name = row_2['ROI_NAME']
    
    mean_ad = np.mean(df_ad[roi_name])
    mean_lewy = np.mean(df_lewy[roi_name])
    
    roi_voxels = np.where(atlas_data == roi_num)

    ad_pattern[roi_voxels] = mean_ad
    lewy_body_pattern[roi_voxels] = mean_lewy


ad_pat_img = nib.Nifti1Image(ad_pattern, atlas_img.affine, atlas_img.header)
nib.save(ad_pat_img, 'ad_pattern.nii.gz')

lewy_pat_img = nib.Nifti1Image(lewy_body_pattern, atlas_img.affine, atlas_img.header)
nib.save(lewy_pat_img, 'lewy_pattern.nii.gz')
    
    


In [12]:
df_ad = df_pacientes.loc[df_pacientes['Grupo']=='AD']
df_lewy = df_pacientes.loc[df_pacientes['Grupo']=='LEWY']

roi_list = []

for index_2, row_2 in df_atlas.iterrows():

    roi_name = row_2['ROI_NAME']
    t, p = ttest_ind(df_ad[roi_name], df_lewy[roi_name], axis=0, equal_var=True)
    if p < 0.01:
        print(roi_name, t, p)
        roi_list.append(roi_name)

Temporal_Pole -2.9044114080274825 0.006423868599824267
Middle_Temporal_Gyrus_anterior_division -2.9449405299732736 0.005791246618907714
Inferior_Temporal_Gyrus_anterior_division -3.648013636244977 0.0008766686428361144
Intracalcarine_Cortex 3.4228607548346344 0.0016316623398195012
Frontal_Medial_Cortex -2.780857134284928 0.008775985499674939
Supracalcarine_Cortex 3.0143807283736326 0.004841642830535394


In [15]:
df_pacientes

,Unnamed: 0,SUBJECT_ID,ORIG_GROUP,LAST_DIAGNOSIS,FDG_ID,FDG_AGE,AGE_DEATH,ALZHEIMER PATHOLOGY,LEWY PATHOLOGY,AD_Pos,...,Planum_Polare,Heschl's_Gyrus,Planum_Temporale,Supracalcarine_Cortex,Occipital_Pole,Thalamus,Striatum,Brainstem,Hippocampus,Amygdala
0,0,003_S_4892,AD,AD,334485,75.3,76,3,0,1.0,...,-0.112321,-0.958489,-1.922645,2.093158,0.357431,0.006967,-0.296966,-0.278695,-0.094379,-0.625237
1,1,005_S_0448,MCI,AD,226091,90.6,93,1,2,0.0,...,-0.080783,0.581598,0.430449,-1.002594,-2.052067,-1.566412,1.811901,1.976478,0.854247,1.853367
2,2,005_S_4910,AD,AD,347770,81.7,84,3,4,1.0,...,-3.597921,-2.232594,-2.430986,-0.353698,-0.505277,-1.481930,-2.080433,0.162197,-2.105653,-1.938893
3,3,006_S_4546,AD,AD,300098,71.2,75,3,2,1.0,...,0.937891,1.258248,0.175496,2.846217,1.965820,-1.520694,-1.832788,0.432951,-0.682365,0.846385
4,4,011_S_0008,CN,MCI,26885,85.6,95,1,3,0.0,...,1.075426,0.278105,-0.128056,-0.168973,-0.532507,0.968607,0.803972,0.301074,0.950026,0.608191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,68,127_S_4500,AD,AD,417020,92.5,95,3,0,1.0,...,0.745240,-0.208900,-0.339694,-2.179645,-1.124119,-1.936402,-1.862072,0.079512,0.819908,1.461137
69,69,127_S_4765,EMCI,AD,871795,81.1,81,3,3,1.0,...,1.722327,-0.339546,-0.589497,-1.592252,-3.282584,1.862965,1.764669,1.218454,1.584905,1.544832
70,70,127_S_5058,AD,AD,360030,61.9,66,2,3,1.0,...,2.331087,0.977442,0.349620,0.075130,0.411157,-0.471721,-0.887564,-0.471022,0.377695,0.904273
71,71,129_S_1246,MCI,MCI,362092,78.5,80,2,0,1.0,...,-1.454552,-0.287481,-0.080177,0.642108,0.269712,-0.247182,1.028909,-1.428884,-0.627655,-0.710637


In [30]:
from sklearn.linear_model import LogisticRegression

df_train = df_pacientes.loc[df_pacientes['Grupo'] != 'Negative']
df_train = df_train.loc[df_train['Grupo'] != 'Mixed']

X = df_train[roi_list]
y = df_train['Grupo']

clf = LogisticRegression(random_state=0, class_weight='balanced').fit(X, y)

In [31]:
clf.score(X, y)

0.8888888888888888

In [32]:
from  sklearn.metrics import confusion_matrix

y_pred = clf.predict(X)
confusion_matrix(y, y_pred, labels=['AD','LEWY'])

array([[25,  3],
       [ 1,  7]], dtype=int64)

In [62]:
def preprocess_pet_img(pet_path):

    features = roi_list
    df_atlas = pd.read_csv('Harvard_Oxford.csv')
    atlas_img = nib.load(atlas_path)
    atlas_data = atlas_img.get_fdata()
    
    pet_img = nib.load(pet_path)
    pet_data = pet_img.get_fdata()
    
    pat_list = []

    for roi in roi_list:
        df_pat = pd.DataFrame
        roi_num = df_atlas.loc[df_atlas['ROI_NAME']==roi]['ROI_NUM'].values[0]
        roi_promedio = df_atlas.loc[df_atlas['ROI_NAME']==roi]['PROMEDIO'].values[0]
        roi_std = df_atlas.loc[df_atlas['ROI_NAME']==roi]['STD'].values[0]
        
        roi_voxel = np.where(atlas_data == roi_num)
        val_patient = np.mean(pet_data[roi_voxel])

        pat_roi_zscore = (val_patient - roi_promedio)/roi_std
        
        pat_list.append(pat_roi_zscore)

        #df_pacientes.loc[index_1,roi_name] = pat_roi_zscore

    return pat_list

def classify_pet_img(pet_path):


    feature_list = preprocess_pet_img(pet_path)
    feature_list = np.array(feature_list).reshape(1,-1)
    out_class = clf.predict(feature_list)[0]
    
    if out_class == 'AD':
        
        print('El patrón de imagen mostrado por el paciente introducido es sugestivo de enfermedad de Alzheimer. Un diagnóstico temprano y un estilo de vida saludable pueden ayudar a manejar sus síntomas y ralentizar su avance')
        
    elif out_class == 'LEWY':

        print('El patrón de imagen mostrado por el paciente introducido es sugestivo de enfermedad de demencia con cuerpos de Lewy.La demencia con cuerpos de Lewy es una enfermedad neurodegenerativa que combina síntomas de la enfermedad de Alzheimer y la enfermedad de Parkinson, como la pérdida de memoria y la lentitud en los movimientos. El diagnóstico temprano y el manejo adecuado de los síntomas pueden mejorar la calidad de vida de los pacientes y ayudar a sus cuidadores a adaptarse a los desafíos')
        
    else:
        print('Se ha detectado un error y su imagen no se ha podido procesar correctamente')
        
        
    
    

In [67]:
pet_path = r'C:\Users\jesus\Work\MIA\Autopsia\005_S_0448_I226091\swfdg_normhist.hdr'
classify_pet_img(pet_path)

El patrón de imagen mostrado por el paciente introducido es sugestivo de enfermedad de demencia con cuerpos de Lewy.La demencia con cuerpos de Lewy es una enfermedad neurodegenerativa que combina síntomas de la enfermedad de Alzheimer y la enfermedad de Parkinson, como la pérdida de memoria y la lentitud en los movimientos. El diagnóstico temprano y el manejo adecuado de los síntomas pueden mejorar la calidad de vida de los pacientes y ayudar a sus cuidadores a adaptarse a los desafíos
